# 0. Configuración de librerias y requisitos previos

In [115]:
"""
Se importan las librerías necesarias para el procesamiento de texto y manejo de archivos.

- nltk.tokenize.regexp_tokenize: para tokenización basada en expresiones regulares.
- nltk.stem.SnowballStemmer: para aplicar stemming en inglés.
- nltk.corpus.stopwords: para cargar listas de stopwords.
- nltk: librería principal para procesamiento de lenguaje natural.

- xml.etree.ElementTree: para parseo de archivos en formato XML.
- collections.defaultdict: para estructuras de datos con valores por defecto.
- os: para operaciones relacionadas con el sistema de archivos.
- re: para trabajar con expresiones regulares.

- typing: para anotaciones de tipo.
"""

from nltk.tokenize import regexp_tokenize
from nltk.stem import SnowballStemmer

import xml.etree.ElementTree as ET

from collections import defaultdict
import os

from typing import List, Dict, Union, Set

In [116]:
"""
Carga de stopwords desde un archivo local.

- Evita la descarga de NLTK y permite usar una lista personalizada de palabras vacías.
- Se construye el conjunto `stop_words` para filtrar palabras vacías durante el preprocesamiento.
"""

# NO cambie esta ruta
path_stopwords = "../data/stopwords/english"

def load_stopwords(file_path: str) -> Set[str]:
    
    """
    Carga stopwords desde un archivo de texto plano.

    Cada línea del archivo debe contener una palabra. La función convierte
    todas las palabras a minúsculas y elimina líneas vacías.

    Parámetros
    ----------
    file_path : str
        Ruta del archivo de stopwords.

    Retorna
    -------
    Set[str]
        Conjunto de palabras que deben considerarse stopwords.
    """
    
    with open(file_path, "r", encoding = "utf-8") as file:
        stop_words = {line.strip().lower() for line in file if line.strip()}
    return stop_words

stop_words = load_stopwords(path_stopwords)

In [117]:
"""
Configuración del stemmer y definición del patrón de tokenización.

- Se inicializa `SnowballStemmer("english")` para reducir las palabras a su raíz 
  durante el preprocesamiento.
- Se define la expresión regular `pattern` que controla cómo se segmenta el texto
  en tokens. Incluye reglas para:
    * Abreviaturas (e.g., U.S.A.)
    * Palabras con guiones internos (e.g., non-euclidean)
    * Monedas, números y porcentajes
    * Puntos suspensivos (...)
    * Signos de puntuación como tokens separados
"""

stemmer = SnowballStemmer("english")

pattern = r'''(?x)              
    (?:[A-Z]\.)+[A-Z]?          # abreviaturas, e.g. U.S.A.
  | [A-Za-z]+(?:-[A-Za-z]+)*    # palabras con guiones internos, e.g. non-euclidean
  | \$?\d+(?:\.\d+)?%?          # monedas, números, porcentajes
  | \.\.\.                      # puntos suspensivos
  | [][.,;"'?():_`-]            # puntuación como tokens separados
'''

# 1. Preprocesamiento del texto

In [118]:
def preprocess(text: str) -> List[str]:
    
	"""
    Preprocesa un texto aplicando tokenización, normalización, eliminación de stopwords y stemming.

    El proceso de preprocesamiento transforma un texto crudo en una lista de tokens
    limpios y estandarizados, listos para tareas de recuperación de información o NLP.

    Pasos:
        1. Tokenización: se divide el texto en palabras usando la expresión regular `pattern`.
        2. Normalización: se convierten todos los tokens a minúsculas.
        3. Eliminación de stopwords: se filtran palabras vacías presentes en `stop_words`.
        4. Stemming: se aplica `SnowballStemmer` para reducir los tokens a su raíz.

    Parámetros
    ----------
    text : str
        Texto de entrada a procesar.

    Retorna
    -------
    List[str]
        Lista de tokens preprocesados.
    """
      
	# Generación de tokens
	tokens = regexp_tokenize(text, pattern)
	# Normalización 
	tokens = [token.lower() for token in tokens]
	# Eliminación de palabras de parada
	tokens = [token for token in tokens if token not in stop_words]
	# Stemming 
	tokens = [stemmer.stem(token) for token in tokens]
	return tokens

In [119]:
"""
Ejemplo de ejecución del preprocesamiento de texto.

Se procesa un texto de ejemplo que incluye números, signos de puntuación y abreviaturas. 
"""

example_text = "Se deben invertir $12.000 millones en D.I.A ..."

print("--------** preprocess **--------\n")
print("Parametros:\n")
print("- Texto de entrada = ", example_text)
print("\nResultado:\n")
print("- Tokens procesados = ", preprocess(example_text))


--------** preprocess **--------

Parametros:

- Texto de entrada =  Se deben invertir $12.000 millones en D.I.A ...

Resultado:

- Tokens procesados =  ['se', 'deben', 'invertir', '$12.000', 'millon', 'en', 'd.i.a', '...']


# 2. Carga de documentos

In [120]:
"""
Ruta de los documentos a procesar.

- `path_docs` indica la carpeta donde se encuentran los archivos de texto crudos.
- Cambie esta ruta según la ubicación de sus documentos en su sistema.
"""

# Cambie esta ruta según la ubicación de los archivos en su sistema
path_docs = "../data/docs-raw-texts"

In [121]:
def load_documents(path_docs: str) -> Dict[str, List[str]]:

	"""
    Carga y preprocesa los documentos de una carpeta específica.

    La función recorre todos los archivos con extensión `.naf` en la ruta
    especificada, extrae el ID del documento, el título y el contenido crudo,
    concatena título y contenido, y aplica la función `preprocess` para 
    obtener los tokens preprocesados de cada documento.

    Parámetros
    ----------
    path_docs : str
        Ruta de la carpeta que contiene los archivos `.naf` a procesar.
        Cambie esta ruta según la ubicación de sus documentos.

    Retorna
    -------
    Dict[str, List[str]]
        Diccionario donde las claves son los `doc_id` de cada documento y
        los valores son listas de tokens preprocesados.
    """
	
	docs = {}
	for file_name in os.listdir(path_docs):
		# Solo se revisan los archivos con extensión correcta
		if not file_name.endswith(".naf"):
			continue
		tree = ET.parse(os.path.join(path_docs, file_name))
		root = tree.getroot()
		# Id recuperado desde el atributo pupblicId
		doc_id = root.find("nafHeader/public").attrib["publicId"]
		# Titulo desde el atributo title
		title = root.find("nafHeader/fileDesc").attrib.get("title", "")
		# Contenido desde <raw>
		raw_element = root.find("raw")
		content = raw_element.text if raw_element is not None else ""
		# concatenar titulo + contenido para preprocesar
		tokens = preprocess(title +  " " + content)
		docs[doc_id] = tokens
	return docs

In [122]:
"""
Ejemplo de carga y preprocesamiento de documentos.

Se utiliza la función `load_documents` para cargar todos los archivos `.naf`
de la carpeta especificada en `path_docs`. Cada documento se preprocesa y se almacena
en un diccionario donde la clave es el `doc_id` y el valor es la lista de tokens.

Luego se muestra el contenido preprocesado de un documento específico
con ID "d006".
"""

documents = load_documents(path_docs)
example_id = "d006"

print("--------** load documents **--------\n")
print("Parametros:\n")
print("- Ruta de los documentos = ", path_docs)
print("\nResultado:\n")
print("- Documentos cargados = ", len(documents))
print("- Ejemplo documento " + example_id + " = ", documents[example_id])

--------** load documents **--------

Parametros:

- Ruta de los documentos =  ../data/docs-raw-texts

Resultado:

- Documentos cargados =  331
- Ejemplo documento d006 =  ['eugenio', 'beltrami', 'non-euclidian', 'geometri', 'eugenio', 'beltrami', 'non-euclidian', 'geometri', '.', 'eugenio', 'beltrami', '(', '1835', '-', '1900', ')', '.', 'novemb', '16', ',', '1835', ',', 'italian', 'mathematician', 'eugenio', 'beltrami', 'born', '.', 'notabl', 'work', 'concern', 'differenti', 'geometri', 'mathemat', 'physic', '.', 'work', 'note', 'especi', 'clariti', 'exposit', '.', 'first', 'prove', 'consist', 'non-euclidean', 'geometri', 'model', 'surfac', 'constant', 'curvatur', ',', 'pseudospher', '.', 'eugenio', 'beltrami', 'born', 'cremona', 'lombardi', ',', 'part', 'austrian', 'empir', ',', 'part', 'itali', '.', 'son', 'artist', 'paint', 'miniatur', ',', 'young', 'eugenio', 'certain', 'inherit', 'artist', 'talent', 'famili', ',', 'case', 'addit', 'mathemat', 'talent', 'would', 'acquir', ',', 'm

# 3. Construcción de índice invertido

In [123]:
def build_inverted_index(docs: Dict[str, List[str]]) -> Dict[str, Dict[str, Union[int, List[str]]]]:
    
	"""
    Construye un índice invertido a partir de un conjunto de documentos preprocesados.

    El índice invertido permite recuperar rápidamente qué documentos contienen
    cada token. Para cada token, se almacena:
        - 'df': la frecuencia de documentos (número de documentos que contienen el token).
        - 'postings': la lista ordenada de IDs de documentos donde aparece el token.

    Parámetros
    ----------
    docs : Dict[str, List[str]]
        Diccionario de documentos preprocesados. Las claves son `doc_id` y los
        valores son listas de tokens obtenidos con la función `preprocess`.

    Retorna
    -------
    Dict[str, Dict[str, Union[int, List[str]]]]
        Diccionario donde cada clave es un token y su valor es un diccionario
        con la frecuencia de documentos ('df') y la lista de postings ('postings').
    """
     
	inverted = defaultdict(set)
	for doc_id, tokens in docs.items():
		for token in tokens:
			inverted[token].add(doc_id)
	# Convertir sets a listas ordenadas
	for token in inverted:
		inverted[token] = {"df" : len(inverted[token]), "postings" : sorted(inverted[token])}
	return inverted

In [124]:
"""
Ejemplo de construcción del índice invertido.

Se utiliza la función `build_inverted_index` para generar un índice invertido
a partir del diccionario `documents` preprocesados. El índice permite consultar
rápidamente en qué documentos aparece cada token y su frecuencia de documentos (df).

Luego se muestra un ejemplo de token presente en el índice.
"""

index = build_inverted_index(documents)
example_token = "play"

print("--------** build inverted index **--------\n")
print("Parámetros:\n")
print("- Documentos = ", documents)
print("\nResultado:\n")
print("- Número de tokens en el índice = ", len(index))
print(f"- Ejemplo token " + example_token + " = ", index.get(example_token))

--------** build inverted index **--------

Parámetros:

- Documentos =  {'d210': ['joseph', 'weizenbaum', 'famous', 'eliza', 'joseph', 'weizenbaum', 'famous', 'eliza', '.', 'joseph', 'weizenbaum', '(', '1923', '-', '2008', ')', 'photo', ':', 'ulrich', 'hansen', '.', 'januari', '8', ',', '1923', ',', 'comput', 'scientist', 'joseph', 'weizenbaum', ',', 'pioneer', 'natur', 'languag', 'process', 'artifici', 'intellig', ',', 'later', 'becam', 'one', 'artifici', 'intellig', 'lead', 'critic', ',', 'born', '.', '1966', 'publish', 'simpl', 'program', 'name', 'eliza', ',', 'involv', 'user', 'convers', 'bore', 'strike', 'resembl', 'one', 'psychologist', '.', 'joseph', 'weizenbaum', 'born', 'berlin', 'jewish', 'parent', 'januari', '8', ',', '1923', '.', 'abl', 'escap', 'nazi', 'germani', 'januari', '1936', ',', 'emigr', 'famili', 'unit', 'state', ',', 'start', 'studi', 'mathemat', 'wayn', 'state', 'univers', 'detroit', '1941', '.', 'howev', ',', 'studi', 'interrupt', 'war', ',', 'serv', 'militari

# 4. Definición del los algoritmos de mezcla y recuperación de consultas

In [125]:
def merge_and(postings_1: List[str], postings_2: List[str]) -> List[str]:

	"""
    Realiza la operación AND entre dos listas de postings ordenadas.

    La función devuelve la intersección de dos listas de IDs de documentos,
    utilizando un algoritmo eficiente de dos punteros que aprovecha que las
    listas están ordenadas.

    Parámetros
    ----------
    postings_1 : List[str]
        Primera lista ordenada de IDs de documentos.
    postings_2 : List[str]
        Segunda lista ordenada de IDs de documentos.

    Retorna
    -------
    List[str]
        Lista de IDs de documentos que aparecen en ambas listas.
    """
	
	# Inicialización de los punteros
	i, j = 0, 0
	result = []
	while i < len(postings_1) and j < len(postings_2):
		# Si los valores son iguales, se cumple el operador lógico y se agregan los ids
		# al resultado
		if postings_1[i] == postings_2[j]:
			result.append(postings_1[i])
			i += 1
			j += 1
		# En caso contrario, se avanza el puntero de menor valor
		elif postings_1[i] < postings_2[j]:
			i += 1
		else:
			j += 1
	return result

In [126]:
"""
Ejemplo de operación AND entre dos listas de postings.

Se utiliza la función `merge_and` para obtener la intersección de dos listas
de IDs de documentos ordenadas. Esto simula la operación booleana AND
entre dos conjuntos de resultados de búsqueda.
"""

postings_1 = ["d001", "d002", "d004", "d006"]
postings_2 = ["d002", "d003", "d004", "d005"]

result = merge_and(postings_1, postings_2)

print("--------** merge and **--------\n")
print("Parámetros:\n")
print("- postings_1 = ", postings_1)
print("- postings_2 = ", postings_2)
print("\nResultado:\n")
print("- Documentos comunes = ", result)

--------** merge and **--------

Parámetros:

- postings_1 =  ['d001', 'd002', 'd004', 'd006']
- postings_2 =  ['d002', 'd003', 'd004', 'd005']

Resultado:

- Documentos comunes =  ['d002', 'd004']


In [127]:
def merge_not(postings: List[str], all_docs: List[str]) -> List[str]:

	"""
    Realiza la operación NOT entre una lista de postings y el conjunto total de documentos.

    La función devuelve la lista de IDs de documentos que **no** aparecen en `postings`,
    utilizando un algoritmo eficiente de dos punteros que aprovecha que ambas listas
    (`postings` y `all_docs`) están ordenadas.

    Parámetros
    ----------
    postings : List[str]
        Lista ordenada de IDs de documentos que deben excluirse.
    all_docs : List[str]
        Lista ordenada de todos los IDs de documentos en la colección.

    Retorna
    -------
    List[str]
        Lista de IDs de documentos que no aparecen en `postings`.
    """

	# Inicialización de los punteros
	result = []
	i, j = 0, 0
	while i < len(all_docs) and j < len(postings):
		# Si los valores son iguales, no se cumple el operador lógico
		if all_docs[i] == postings[j]:
			i += 1
			j += 1
		# Se cumple el operador y se agrega el id a los resultados
		elif all_docs[i] < postings[j]:
			result.append(all_docs[i])
			i += 1
		else:
			j += 1
	# Se agregan todos los no evaluados porque cumplen el operador
	result.extend(all_docs[i:])
	return result

In [128]:
"""
Ejemplo de operación NOT sobre una lista de postings.

Se utiliza la función `merge_not` para obtener los documentos que **no** aparecen
en la lista de postings proporcionada. Esto simula la operación booleana NOT
sobre un conjunto de resultados de búsqueda.
"""

all_docs = ["d001", "d002", "d003", "d004", "d005", "d006"]
postings = ["d002", "d004"]

result = merge_not(postings, all_docs)

print("--------** merge not **--------\n")
print("Parámetros:\n")
print("- all_docs = ", all_docs)
print("- postings = ", postings)
print("\nResultado:\n")
print("- Documentos que no aparecen en postings = ", result)


--------** merge not **--------

Parámetros:

- all_docs =  ['d001', 'd002', 'd003', 'd004', 'd005', 'd006']
- postings =  ['d002', 'd004']

Resultado:

- Documentos que no aparecen en postings =  ['d001', 'd003', 'd005', 'd006']


In [129]:
def read_and_format_query(file_path: str) -> List[str]:
	
	"""
    Lee una consulta desde un archivo `.naf` y la formatea para evaluación booleana.

    La función realiza los siguientes pasos:
        1. Parsea el archivo XML y obtiene el contenido de la etiqueta <raw>.
        2. Preprocesa el texto usando la función `preprocess` (tokenización,
           normalización, eliminación de stopwords y stemming).
        3. Convierte la lista de tokens en notación postfija (RPN) insertando
           operadores 'AND' entre cada token, lista para ser evaluada con 
           `boolean_query`.

    Parámetros
    ----------
    file_path : str
        Ruta del archivo `.naf` que contiene la consulta.

    Retorna
    -------
    List[str]
        Lista de tokens de la consulta en notación postfija (RPN).
        Retorna lista vacía si la consulta está vacía o no contiene tokens.
    """

	tree = ET.parse(file_path)
	root = tree.getroot()
	query = root.find("raw").text.strip()
	# Tokens procesados pero sin unión por operadores lógicos
	tokens = preprocess(query)
	if not tokens:
		return []
	rpn_tokens = [tokens[0]]
	# Se unen los tokens usando operadores lógicos
	for token in tokens[1:]:
		rpn_tokens.append(token)
		rpn_tokens.append("AND")
	return rpn_tokens

In [130]:
"""
Ejemplo de lectura y formateo de una consulta desde archivo `.naf`.

Se utiliza la función `read_and_format_query` para:
1. Leer el contenido de la etiqueta <raw> de un archivo de consulta.
2. Preprocesar el texto (tokenización, normalización, eliminación de stopwords y stemming).
3. Convertir los tokens en notación postfija (RPN) insertando operadores 'AND' entre ellos.

El resultado es una lista de tokens lista para ser evaluada con `boolean_query`.
"""

# NO cambie esta ruta
query_file = "../data/queries-raw-texts/wes2015.q41.naf"
rpn_tokens = read_and_format_query(query_file)

print("--------** read_and_format_query **--------\n")
print("Parámetros:\n")
print("- Archivo de consulta = ", query_file)
print("\nResultado:\n")
print("- Tokens en notación postfija (RPN) = ", rpn_tokens)


--------** read_and_format_query **--------

Parámetros:

- Archivo de consulta =  ../data/queries-raw-texts/wes2015.q41.naf

Resultado:

- Tokens en notación postfija (RPN) =  ['person', 'hannov', 'AND']


In [131]:
def boolean_query(query_tokens: List[str], index: Dict[str, Dict[str, Union[int, List[str]]]], all_docs: List[str]) -> List[str]:
    
	"""
    Evalúa una consulta booleana en notación postfija (RPN) sobre un índice invertido.

    La función procesa tokens de consulta que pueden ser operadores booleanos
    ('AND', 'NOT') o términos simples. Para cada token:
        - Si es 'AND', realiza la intersección de los dos últimos resultados en la pila.
        - Si es 'NOT', realiza la diferencia con todos los documentos.
        - Si es un término, obtiene su lista de postings desde el índice invertido.
    
    La evaluación se realiza usando una pila (stack) para combinar los resultados
    paso a paso, retornando finalmente la lista de documentos que cumplen la consulta.

    Parámetros
    ----------
    query_tokens : List[str]
        Lista de tokens de la consulta en notación postfija (RPN).
        Ejemplo: ['amor', 'vida', 'AND', 'NOT'].
    index : Dict[str, Dict[str, Union[int, List[str]]]]
        Índice invertido construido con `build_inverted_index`.
        Cada token apunta a un diccionario con:
            - 'df': frecuencia de documentos
            - 'postings': lista de documentos que contienen el token
    all_docs : List[str]
        Lista ordenada de todos los IDs de documentos en la colección,
        usada para la operación NOT.

    Retorna
    -------
    List[str]
        Lista de IDs de documentos que cumplen la consulta booleana.
        Retorna lista vacía si no hay resultados o la consulta está vacía.
    """
     
	stack = []
	for token in query_tokens:		
		if token == "AND":
			postings_1 = stack.pop()
			postings_2 = stack.pop()
			# Se agrega el resultado para seguir realizando operaciones
			stack.append(merge_and(postings_1, postings_2))
		elif token == "NOT":
			postings = stack.pop()
			# Se agrega el resultado para seguir realizando operaciones
			stack.append(merge_not(postings, all_docs))
		else:
			# Se obtienen los postings del índice
			postings = index.get(token, {"postings": []})["postings"]
			stack.append(postings)
	return stack.pop() if stack else []

In [132]:
"""
Ejemplo de evaluación de una consulta booleana sobre un índice invertido.

Se utiliza la función `boolean_query` para procesar una consulta en notación
postfija (RPN) utilizando un índice invertido `index` y la lista de todos
los documentos `all_docs`. La función combina operaciones AND y NOT entre
los tokens de la consulta para retornar los documentos que cumplen la condición.
"""

all_docs_list = sorted(documents.keys())
result_docs = boolean_query(rpn_tokens, index, all_docs_list)

print("--------** boolean_query **--------\n")
print("Parámetros:\n")
print("- Tokens de consulta (RPN) = ", rpn_tokens)
print("- Todos los documentos = ", all_docs_list)
print("\nResultado:\n")
print("- Documentos que cumplen la consulta = ", result_docs)


--------** boolean_query **--------

Parámetros:

- Tokens de consulta (RPN) =  ['person', 'hannov', 'AND']
- Todos los documentos =  ['d001', 'd002', 'd003', 'd004', 'd005', 'd006', 'd007', 'd008', 'd009', 'd010', 'd011', 'd012', 'd013', 'd014', 'd015', 'd016', 'd017', 'd018', 'd019', 'd020', 'd021', 'd022', 'd023', 'd024', 'd025', 'd026', 'd027', 'd028', 'd029', 'd030', 'd031', 'd032', 'd033', 'd034', 'd035', 'd036', 'd037', 'd038', 'd039', 'd040', 'd041', 'd042', 'd043', 'd044', 'd045', 'd046', 'd047', 'd048', 'd049', 'd050', 'd051', 'd052', 'd053', 'd054', 'd055', 'd056', 'd057', 'd058', 'd059', 'd060', 'd061', 'd062', 'd063', 'd064', 'd065', 'd066', 'd067', 'd068', 'd069', 'd070', 'd071', 'd072', 'd073', 'd074', 'd075', 'd076', 'd077', 'd078', 'd079', 'd080', 'd081', 'd082', 'd083', 'd084', 'd085', 'd086', 'd087', 'd088', 'd089', 'd090', 'd091', 'd092', 'd093', 'd094', 'd095', 'd096', 'd097', 'd098', 'd099', 'd100', 'd101', 'd102', 'd103', 'd104', 'd105', 'd106', 'd107', 'd108', '

# 5. Evaluación de las consultas de prueba

In [133]:
def process_queries_folder(path_queries: str, index: Dict[str, Dict[str, Union[int, List[str]]]], all_docs: List[str], output_file: str) -> None:

	"""
    Procesa todos los archivos de consultas en una carpeta y guarda los resultados.

    La función recorre todos los archivos `.naf` en `path_queries`, formatea
    cada consulta, la evalúa sobre el índice invertido `index` y la lista de
    todos los documentos `all_docs`, y escribe los resultados en un archivo
    de salida.

    Cada línea del archivo de salida contiene el ID de la consulta seguido
    de los IDs de documentos que cumplen la consulta, separados por comas.

    Parámetros
    ----------
    path_queries : str
        Ruta de la carpeta que contiene los archivos `.naf` de consultas.
    index : Dict[str, Dict[str, Union[int, List[str]]]]
        Índice invertido construido con `build_inverted_index`.
    all_docs : List[str]
        Lista ordenada de todos los IDs de documentos en la colección.
    output_file : str
        Ruta del archivo donde se guardarán los resultados.
    
    Retorna
    -------
    None
    """
	
	with open(output_file, "w", encoding = "utf-8") as out_file:
		# Recorrido ordenado por id de query
		for fname in sorted(os.listdir(path_queries)):
			# Solo se revisan archivos validos
			if fname.endswith(".naf"):
				file_path = os.path.join(path_queries, fname)
				tree = ET.parse(file_path)
				root = tree.getroot()
				# Extracción del id referido dentro del documento
				public_elem = root.find(".//public")
				query_id = public_elem.attrib.get("publicId")
				# Construcción de la consulta
				query_tokens = read_and_format_query(file_path)
				# Ejecución de la busqueda
				docs = boolean_query(query_tokens, index, all_docs)
				out_file.write(f"{query_id} {','.join(docs)}\n")

In [134]:
"""
Ejemplo de procesamiento de consultas en una carpeta.

Se utiliza la función `process_queries_folder` para:
1. Leer todas las consultas en formato `.naf` desde `path_queries`. Puede cambiar la ruta.
2. Evaluarlas sobre el índice invertido `index`.
3. Guardar los resultados en `output_file`.

Luego se muestra el contenido del archivo de salida para verificar los resultados.
"""

# Cambie esta ruta según la ubicación de los archivos en su sistema
path_queries = "../data/queries-raw-texts"
output_file = "../results/BSII-AND-queries_results.tsv"

process_queries_folder(path_queries, index, all_docs_list, output_file)

print("--------** process queries folder **--------\n")
print("Parámetros:\n")
print("- Carpeta de consultas = ", path_queries)
print("- Archivo de resultados = ", output_file)
print("\nResultado (primeras líneas):\n")

# Mostrar las primeras 5 líneas del archivo de salida
with open(output_file, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i >= 4:
            break


--------** process queries folder **--------

Parámetros:

- Carpeta de consultas =  ../data/queries-raw-texts
- Archivo de resultados =  ../results/BSII-AND-queries_results.tsv

Resultado (primeras líneas):

q01
q02 d291,d293
q03 d105,d147,d152,d283,d291,d318
q04 d286
q06 d026,d029,d069,d257,d297,d303,d329
